In [9]:
import sys
import os
import torch
import glob
import numpy as np
sys.path.insert(1, '..')
from processing.processing_functions import *

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
if use_cuda == True:
    print("Using Cuda")
else:
    print("Using CPU")

Using CPU


In [ ]:
import os
from torchvision import transforms, utils
from torch.utils.data import Dataset
import tifffile
import glob
import yaml

# yaml experiment file location
project_path = os.path.abspath(os.path.abspath(os.path.dirname('__file__')) + "/../")
exp_path = "/config/convnets/ResUNet"
exp_file = "/exp_1.yml"

with open(project_path + exp_path + exp_file, "r") as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

raw_path = "/Users/jasonfung/haaslabdataimages@gmail.com - Google Drive/My Drive/Images/Raw/*.tif"
mask_path = "/Users/jasonfung/haaslabdataimages@gmail.com - Google Drive/My Drive/Images/Mask/*.tif"

raw_filename_list = glob.glob(raw_path) 
mask_filename_list = glob.glob(mask_path)

# Pre Shuffle
raw_filename_list.sort()
mask_filename_list.sort()

# Shuffle the filename list
from sklearn.utils import shuffle
raw_filename_list, mask_filename_list = shuffle(raw_filename_list, mask_filename_list, random_state = 42)

# define patching parameters
lateral_steps = config['DATASET']['lateral_steps']
axial_steps = config['DATASET']['axial_steps']
patch_size = (axial_steps, lateral_steps, lateral_steps)
split_size = config['DATASET']['split_size']

raw_training_list, mask_training_list = raw_filename_list[:int(split_size*len(raw_filename_list))], mask_filename_list[:int(split_size*len(mask_filename_list))]
raw_testing_list, mask_testing_list = raw_filename_list[int(split_size*len(raw_filename_list)):], mask_filename_list[int(split_size*len(mask_filename_list)):]


In [7]:
def get_class_weights(mask_list, classes = 3):
    # calculate balanced class weights
    # mask_list: list()
    # classes: integer num_class - 1


    process_mask = process_masks(classes)
    tot_mask_len = 0
    mask_bin_count = []
    le = preprocessing.LabelEncoder()
    for mask_name in mask_list:
        mask = tifffile.imread(mask_name).astype(np.float16)
        mask = process_mask(mask)
        mask_ind = le.fit_transform(mask.flatten())
        mask_bin_count.append(np.bincount(mask_ind).astype(np.float64))
        tot_mask_len += len(mask.flatten())

    mask_bin_count = np.sum(np.array(mask_bin_count),axis = 0)
    weights = tot_mask_len / (len(le.classes_) * mask_bin_count.astype(np.float64))
    return weights
    
test_weights = get_class_weights(mask_training_list)

[array([9.139044e+06, 9.353000e+03, 2.499600e+04, 1.647000e+03]), array([1.3322707e+07, 5.6730000e+03, 3.9245000e+04, 1.7190000e+03]), array([1.4408384e+07, 1.8920000e+03, 7.0510000e+03, 5.9300000e+02]), array([4.9726276e+07, 1.0850000e+04, 6.6928000e+04, 3.3060000e+03]), array([2.1720128e+07, 8.6580000e+03, 2.8316000e+04, 8.5000000e+02]), array([1.3094437e+07, 2.4280000e+03, 9.6620000e+03, 6.7300000e+02]), array([1.6921378e+07, 2.9145000e+04, 8.4869000e+04, 3.9680000e+03]), array([2.7999546e+07, 1.0398000e+04, 3.8658000e+04, 8.0600000e+02]), array([2.323913e+07, 1.255900e+04, 7.764100e+04, 1.486000e+03]), array([1.8059674e+07, 7.9160000e+03, 1.7935000e+04, 2.4110000e+03]), array([2.0134425e+07, 5.1590000e+03, 4.1832000e+04, 3.6720000e+03]), array([2.2252225e+07, 1.2728000e+04, 1.6971000e+04, 3.1600000e+02]), array([3.876191e+07, 7.543000e+03, 2.290400e+04, 4.955000e+03]), array([1.7023313e+07, 4.4340000e+03, 1.0615000e+04, 9.9800000e+02])]


In [8]:
test_weights

array([2.50526286e-01, 5.95106140e+02, 1.57112327e+02, 2.79604321e+03])